In [1]:
base_directories = {}
base_directories["delta_qm"] = "/project/def-psavard/ladamek/sagitta_bias_matrices/Injection_Dec17_inject_{inject}_method_{method}_region_{detector_location}_{end_string}_round_{round}/OutputFiles"
base_directories["matrix"] = "/project/def-psavard/ladamek/sagitta_bias_matrices/Injection_Dec17_inject_{inject}_region_{detector_location}_{end_string}_round_{round}/OutputFiles"

import matplotlib.pyplot as plt
from atlasify import atlasify
plt.rcParams.update({'font.size': 15})

method = "delta_qm"

def calc_convergence_mse(injected, measured = 0.0):
    n = 0
    mse = 0

    for i in range(1, injected.GetNbinsX() + 1):
        for j in range(1, injected.GetNbinsY() + 1):
            injected_value = injected.GetBinContent(i, j)
            if type(measured) != float: measured_value = measured.GetBinContent(i, j)
            else: measured_value = measured
            mse += (measured_value - injected_value)**2
            print(measured_value - injected_value)
            n+=1

    return mse / float(n)

def calc_convergence_mae(injected, measured = 0.0):
    n = 0
    mse = 0

    for i in range(1, injected.GetNbinsX() + 1):
        for j in range(1, injected.GetNbinsY() + 1):
            injected_value = injected.GetBinContent(i, j)
            if type(measured) != float: measured_value = measured.GetBinContent(i, j)
            else: measured_value = measured
            mse += abs(measured_value - injected_value)
            n+=1

    return mse / float(n)

def calc_convergence_lad(injected, measured = 0.0):
    lads = []

    for i in range(1, injected.GetNbinsX() + 1):
        for j in range(1, injected.GetNbinsY() + 1):
            injected_value = injected.GetBinContent(i, j)
            if type(measured) != float: measured_value = measured.GetBinContent(i, j)
            else: measured_value = measured
            lads.append(abs(measured_value - injected_value))

    return max(lads)

from MatrixInversion import get_histogram_function
from BiasInjection import solution_histogram
def get_solution_histogram(bias, region = None):
    histogram_function = get_histogram_function(bias)
    histogram = histogram_function(detector_location = region)
    histogram = solution_histogram(histogram)
    return histogram

metrics = [("LAD", calc_convergence_lad), ("MAE", calc_convergence_mae), ("MSE", calc_convergence_mse)]

biases = ["Random", "Global" , "Local", "GlobalPlusLocal"]

for method in ["matrix","delta_qm"][::-1]:

    if "matrix" == method:
        from MatrixInversion import get_deltas_from_job
        global get_deltas_from_job
        rounds = list(range(1, 6))

    elif "delta_qm" == method:
        from DeltaQMIterativeMethod import get_deltas_from_job
        global get_deltas_from_job
        rounds = list(range(1, 17))
        
    for (metric_name, metric_function) in metrics:
        for region in ["ME", "ID"]:#, "ME"]: 
            for end_string in ["tight_preselection", "loose_preselection_tight_select_after_correction", "loose_preselection_tight_select_before_correction"]: #, "loose_preselection_tight_select_after_correction"]:
                plt.figure()
                measured_set = {}
                for bias in biases:
                    target = get_solution_histogram(bias, region=region)
                    target.Scale(1000.0)
                    measured_set[bias] = [metric_function(target, measured = 0.0)]
                    for r in rounds:
                        injection = get_solution_histogram(bias, region=region)
                        directory = base_directories[method].format(inject = bias, method = method, detector_location = region, round = r, end_string = end_string)
                        subtraction_dir = base_directories[method].format(inject = "None", method = method, detector_location = region, round = r, end_string = end_string)
                        measured = get_deltas_from_job(directory)[0]
                        measured.Scale(1000.0)
                        subtraction = get_deltas_from_job(subtraction_dir)[0]
                        subtraction.Scale(1000.0)
                        measured.Add(subtraction, -1.0)
                        mse = metric_function(target, measured)
                        measured_set[bias].append(mse)


                x = [0] + rounds
                maxs = []
                mins = []
                for bias in biases:
                    y = measured_set[bias]
                    maxs.append(max(y))
                    mins.append(min(y))
                    plt.plot(x, y, label=bias)

                # return the current ylim
                bottom, top = plt.ylim()
                plt.ylim(min(mins)/10.0, max(maxs)*50.0)

                plt.xlabel("Number of Iterations")
                if metric_name == "MSE": num = -2
                else: num = -1
                plt.ylabel("{name}   [$TeV^{num}$]".format(name = metric_name, num = num))
                plt.yscale("log")
                if method == "matrix": plt.xticks(x)
                else: plt.xticks([xi for i,xi in enumerate(x) if i % 2 == 0])
                plt.legend()
                atlasify("Internal")

                import os
                savedir = os.path.join(os.getenv("MomentumValidationDir"), "ConvergencePlots")
                if not os.path.exists(savedir): os.makedirs(savedir)
                plt.tight_layout()
                plt.savefig(os.path.join(savedir, "{}_{}_{}_{}.png".format(method, end_string, region, metric_name)))
            
            
            

Welcome to JupyROOT 6.14/04
Trying to open /project/def-psavard/ladamek/sagitta_bias_matrices/Injection_Dec17_inject_Random_method_delta_qm_region_ME_tight_preselection_round_1/OutputFiles/CACHE.pkl
Successfully opened /project/def-psavard/ladamek/sagitta_bias_matrices/Injection_Dec17_inject_Random_method_delta_qm_region_ME_tight_preselection_round_1/OutputFiles/CACHE.pkl
Trying to open /project/def-psavard/ladamek/sagitta_bias_matrices/Injection_Dec17_inject_None_method_delta_qm_region_ME_tight_preselection_round_1/OutputFiles/CACHE.pkl
Successfully opened /project/def-psavard/ladamek/sagitta_bias_matrices/Injection_Dec17_inject_None_method_delta_qm_region_ME_tight_preselection_round_1/OutputFiles/CACHE.pkl
Trying to open /project/def-psavard/ladamek/sagitta_bias_matrices/Injection_Dec17_inject_Random_method_delta_qm_region_ME_tight_preselection_round_2/OutputFiles/CACHE.pkl
Successfully opened /project/def-psavard/ladamek/sagitta_bias_matrices/Injection_Dec17_inject_Random_method_del

Trying to open /project/def-psavard/ladamek/sagitta_bias_matrices/Injection_Dec17_inject_Random_method_delta_qm_region_ME_tight_preselection_round_15/OutputFiles/CACHE.pkl
Successfully opened /project/def-psavard/ladamek/sagitta_bias_matrices/Injection_Dec17_inject_Random_method_delta_qm_region_ME_tight_preselection_round_15/OutputFiles/CACHE.pkl
Trying to open /scratch/ladamek/sagittabias_matrices/Injection_Dec17_inject_Random_method_delta_qm_region_ME_tight_preselection_round_14/OutputFiles/CACHE.pkl
Successfully opened /scratch/ladamek/sagittabias_matrices/Injection_Dec17_inject_Random_method_delta_qm_region_ME_tight_preselection_round_14/OutputFiles/CACHE.pkl
Trying to open /scratch/ladamek/sagittabias_matrices/Injection_Dec17_inject_Random_method_delta_qm_region_ME_tight_preselection_round_13/OutputFiles/CACHE.pkl
Successfully opened /scratch/ladamek/sagittabias_matrices/Injection_Dec17_inject_Random_method_delta_qm_region_ME_tight_preselection_round_13/OutputFiles/CACHE.pkl
Tryin

Successfully opened /scratch/ladamek/sagittabias_matrices/Injection_Dec17_inject_Global_method_delta_qm_region_ME_tight_preselection_round_13/OutputFiles/CACHE.pkl
Trying to open /scratch/ladamek/sagittabias_matrices/Injection_Dec17_inject_Global_method_delta_qm_region_ME_tight_preselection_round_12/OutputFiles/CACHE.pkl
Successfully opened /scratch/ladamek/sagittabias_matrices/Injection_Dec17_inject_Global_method_delta_qm_region_ME_tight_preselection_round_12/OutputFiles/CACHE.pkl
Trying to open /scratch/ladamek/sagittabias_matrices/Injection_Dec17_inject_Global_method_delta_qm_region_ME_tight_preselection_round_11/OutputFiles/CACHE.pkl
Successfully opened /scratch/ladamek/sagittabias_matrices/Injection_Dec17_inject_Global_method_delta_qm_region_ME_tight_preselection_round_11/OutputFiles/CACHE.pkl
Trying to open /scratch/ladamek/sagittabias_matrices/Injection_Dec17_inject_Global_method_delta_qm_region_ME_tight_preselection_round_10/OutputFiles/CACHE.pkl
Successfully opened /scratch/la

Trying to open /project/def-psavard/ladamek/sagitta_bias_matrices/Injection_Dec17_inject_Local_method_delta_qm_region_ME_tight_preselection_round_1/OutputFiles/CACHE.pkl
Successfully opened /project/def-psavard/ladamek/sagitta_bias_matrices/Injection_Dec17_inject_Local_method_delta_qm_region_ME_tight_preselection_round_1/OutputFiles/CACHE.pkl
Trying to open /project/def-psavard/ladamek/sagitta_bias_matrices/Injection_Dec17_inject_None_method_delta_qm_region_ME_tight_preselection_round_1/OutputFiles/CACHE.pkl
Successfully opened /project/def-psavard/ladamek/sagitta_bias_matrices/Injection_Dec17_inject_None_method_delta_qm_region_ME_tight_preselection_round_1/OutputFiles/CACHE.pkl
Trying to open /project/def-psavard/ladamek/sagitta_bias_matrices/Injection_Dec17_inject_Local_method_delta_qm_region_ME_tight_preselection_round_2/OutputFiles/CACHE.pkl
Successfully opened /project/def-psavard/ladamek/sagitta_bias_matrices/Injection_Dec17_inject_Local_method_delta_qm_region_ME_tight_preselecti

Trying to open /project/def-psavard/ladamek/sagitta_bias_matrices/Injection_Dec17_inject_Local_method_delta_qm_region_ME_tight_preselection_round_6/OutputFiles/CACHE.pkl
Successfully opened /project/def-psavard/ladamek/sagitta_bias_matrices/Injection_Dec17_inject_Local_method_delta_qm_region_ME_tight_preselection_round_6/OutputFiles/CACHE.pkl
Trying to open /scratch/ladamek/sagittabias_matrices/Injection_Dec17_inject_Local_method_delta_qm_region_ME_tight_preselection_round_5/OutputFiles/CACHE.pkl
Successfully opened /scratch/ladamek/sagittabias_matrices/Injection_Dec17_inject_Local_method_delta_qm_region_ME_tight_preselection_round_5/OutputFiles/CACHE.pkl
Trying to open /scratch/ladamek/sagittabias_matrices/Injection_Dec17_inject_Local_method_delta_qm_region_ME_tight_preselection_round_4/OutputFiles/CACHE.pkl
Successfully opened /scratch/ladamek/sagittabias_matrices/Injection_Dec17_inject_Local_method_delta_qm_region_ME_tight_preselection_round_4/OutputFiles/CACHE.pkl
Trying to open /s

...

Trying to open /project/def-psavard/ladamek/sagitta_bias_matrices/Injection_Dec17_inject_GlobalPlusLocal_method_delta_qm_region_ME_tight_preselection_round_16/OutputFiles/CACHE.pkl
Successfully opened /project/def-psavard/ladamek/sagitta_bias_matrices/Injection_Dec17_inject_GlobalPlusLocal_method_delta_qm_region_ME_tight_preselection_round_16/OutputFiles/CACHE.pkl
Trying to open /scratch/ladamek/sagittabias_matrices/Injection_Dec17_inject_GlobalPlusLocal_method_delta_qm_region_ME_tight_preselection_round_15/OutputFiles/CACHE.pkl
Successfully opened /scratch/ladamek/sagittabias_matrices/Injection_Dec17_inject_GlobalPlusLocal_method_delta_qm_region_ME_tight_preselection_round_15/OutputFiles/CACHE.pkl
Trying to open /scratch/ladamek/sagittabias_matrices/Injection_Dec17_inject_GlobalPlusLocal_method_delta_qm_region_ME_tight_preselection_round_14/OutputFiles/CACHE.pkl
Successfully opened /scratch/ladamek/sagittabias_matrices/Injection_Dec17_inject_GlobalPlusLocal_method_delta_qm_region_ME_t

/cvmfs/sft.cern.ch/lcg/views/LCG_94python3/x86_64-centos7-gcc8-opt/lib/python3.6/site-packages/ipykernel_launcher.py:124: UserWarning: AutoMinorLocator does not work with logarithmic scale
/cvmfs/sft.cern.ch/lcg/views/LCG_94python3/x86_64-centos7-gcc8-opt/lib/python3.6/site-packages/ipykernel_launcher.py:125: UserWarning: AutoMinorLocator does not work with logarithmic scale


Trying to open /project/def-psavard/ladamek/sagitta_bias_matrices/Injection_Dec17_inject_Random_method_delta_qm_region_ME_loose_preselection_tight_select_after_correction_round_1/OutputFiles/CACHE.pkl
Successfully opened /project/def-psavard/ladamek/sagitta_bias_matrices/Injection_Dec17_inject_Random_method_delta_qm_region_ME_loose_preselection_tight_select_after_correction_round_1/OutputFiles/CACHE.pkl
Trying to open /project/def-psavard/ladamek/sagitta_bias_matrices/Injection_Dec17_inject_None_method_delta_qm_region_ME_loose_preselection_tight_select_after_correction_round_1/OutputFiles/CACHE.pkl
Successfully opened /project/def-psavard/ladamek/sagitta_bias_matrices/Injection_Dec17_inject_None_method_delta_qm_region_ME_loose_preselection_tight_select_after_correction_round_1/OutputFiles/CACHE.pkl
Trying to open /project/def-psavard/ladamek/sagitta_bias_matrices/Injection_Dec17_inject_Random_method_delta_qm_region_ME_loose_preselection_tight_select_after_correction_round_2/OutputFiles

Successfully opened /project/def-psavard/ladamek/sagitta_bias_matrices/Injection_Dec17_inject_None_method_delta_qm_region_ME_loose_preselection_tight_select_after_correction_round_14/OutputFiles/CACHE.pkl
Trying to open /scratch/ladamek/sagittabias_matrices/Injection_Dec17_inject_None_method_delta_qm_region_ME_loose_preselection_tight_select_after_correction_round_13/OutputFiles/CACHE.pkl
Successfully opened /scratch/ladamek/sagittabias_matrices/Injection_Dec17_inject_None_method_delta_qm_region_ME_loose_preselection_tight_select_after_correction_round_13/OutputFiles/CACHE.pkl
Trying to open /scratch/ladamek/sagittabias_matrices/Injection_Dec17_inject_None_method_delta_qm_region_ME_loose_preselection_tight_select_after_correction_round_12/OutputFiles/CACHE.pkl
Successfully opened /scratch/ladamek/sagittabias_matrices/Injection_Dec17_inject_None_method_delta_qm_region_ME_loose_preselection_tight_select_after_correction_round_12/OutputFiles/CACHE.pkl
Trying to open /scratch/ladamek/sagit

ValueError: Call this function with update_cache True if there is no cache file already

In [ ]:
delta_qm_data_dir = "/scratch/ladamek/sagittabias_matrices/Data_Dec17_inject_None_method_delta_qm_region_{detector_location}_loose_preselection_tight_select_after_correction_round_{round}/OutputFiles"
delta_qm_mc_dir = base_directories["delta_qm"].format(inject="None", end_string="loose_preselection_tight_select_after_correction", method="delta_qm", detector_location="{detector_location}", round="{round}")
matrix_data_dir = "/scratch/ladamek/sagittabias_matrices/Injection_Dec17_Data_inject_None_region_{detector_location}_tight_select_after_correction_round_{round}/OutputFiles"
matrix_mc_dir = base_directories["matrix"].format(inject="None", end_string="loose_preselection_tight_select_after_correction", method="delta_qm", detector_location="{detector_location}", round="{round}")

rounds = list(range(1, 8))
import MatrixInversion
import DeltaQMIterativeMethod
for region in ["ME", "ID"]:#, "ME"]:   
    for (metric_name, metric_function) in metrics:
        plt.figure()
        y = []
        for r in rounds:

            base_directory = delta_qm_data_dir
            base_subtraction_dir = delta_qm_mc_dir
            directory = base_directory.format(detector_location=region, round=3*r, method=method)
            sagitta_hist = DeltaQMIterativeMethod.get_deltas_from_job(directory)[0]
            subtraction_dir = base_subtraction_dir.format(detector_location=region, round=3*r, method=method)
            sagitta_hist_sub = DeltaQMIterativeMethod.get_deltas_from_job(subtraction_dir)[0]
            sagitta_hist.Add(sagitta_hist_sub, -1.0)
            sagitta_hist.Scale(1000.0)
            sagitta_hist_matrix = sagitta_hist

            base_directory = matrix_data_dir
            base_subtraction_dir = matrix_mc_dir
            directory = base_directory.format(detector_location=region, round=r, method=method)
            sagitta_hist = MatrixInversion.get_deltas_from_job(directory)[0]
            subtraction_dir = base_subtraction_dir.format(detector_location=region, round=r, method=method)
            sagitta_hist_sub = MatrixInversion.get_deltas_from_job(subtraction_dir)[0]
            sagitta_hist.Add(sagitta_hist_sub, -1.0)
            sagitta_hist.Scale(1000.0)
            sagitta_hist_delta_qm = sagitta_hist
            y.append(metric_function(sagitta_hist_matrix, sagitta_hist_delta_qm))
            
        x = rounds
        maxs = [max(y)]
        plt.plot(x, y)

        # return the current ylim
        bottom, top = plt.ylim()
        #plt.ylim(bottom, max(maxs)*5)

        plt.xlabel("Number of Iterations")
        if metric_name == "MSE": num = -2
        else: num = -1
        plt.ylabel("{name}   [$TeV^{num}$]".format(name = metric_name, num = num))
        #plt.yscale("log")
        plt.xticks(x)
        plt.legend()
        atlasify("Internal")

        import os
        savedir = os.path.join(os.getenv("MomentumValidationDir"), "ConvergencePlots")
        if not os.path.exists(savedir): os.makedirs(savedir)
        plt.tight_layout()
        plt.savefig(os.path.join(savedir, "{}_{}_{}.png".format("DataCompareMethods", region, metric_name)))
            

In [ ]:
os.getenv("MomentumValidationDir")

In [ ]:
os.path.join(os.getenv("MomentumValidationDir"), "ConvergencePlots")